In [1]:
import pandas as pd
import sqlalchemy as sa
from redshift_credentials import database, username, password, host, port, schema
from sqlalchemy.orm import sessionmaker
import itertools

In [2]:
def sql_con(in_username, in_password, in_host, in_port, in_database, in_schema):
    '''Create connection function'''
    connection = "redshift+psycopg2://%s:%s@%s:%s/%s" % (in_username, in_password, in_host, str(in_port), in_database)
    engine = sa.create_engine(connection)
    session = sessionmaker()
    session.configure(bind=engine)
    s = session()
    setpath = "SET search_path TO %s" % in_schema
    s.execute(setpath)
    return engine

In [3]:
# Define connection.
conn = sql_con(username, password, host, port, database, schema)

In [26]:
# Primary forecast query.
forecast_query = "SELECT fulfillment_week_ended week_ended, %s, %s qty_returned, COUNT(*) qty_sold, %s * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1%s"

# Actual primary query.
actual_query = "SELECT fulfillment_week_ended week_ended, %s, %s qty_returned, COUNT(*) qty_sold, %s * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1%s"

In [27]:
# Forecasted qty returned/exchanged query.
forecast_qty = "SUM(CASE WHEN send_back__type IN %s AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END)"

# Actual qty returned/exchanged query.
actual_qty = "SUM(CASE WHEN send_back__type in %s THEN 1 ELSE 0 END)"

In [28]:
product_categories = "CASE WHEN shape = 'Rectangle' AND size NOT ILIKE ('%9%x%12%') AND texture = 'Chenile' AND purpose != 'Outdoor' THEN 'rectangular_chenille' WHEN texture = 'Shag' THEN 'shag' WHEN shape = 'Round' THEN 'round' WHEN purpose = 'Outdoor' THEN 'outdoor' WHEN texture = 'Berber' THEN 'plush' WHEN size ILIKE ('%9%x%12%') THEN '9x12' WHEN texture = 'Rubber' THEN 'rug_pad' END product"

product_cushioned = "CASE WHEN product_sub_type = 'cushioned' THEN 'cushioned' ELSE 'classic' END product"

In [43]:
query_params = {
    'return_product_weekly': {'view': {'actual': [actual_query, actual_qty], 'forecast': [forecast_query, forecast_qty]},
                              'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                              'send_back__type': {'send_back_both': "('return', 'exchange')",
                                                  'return': "('return')", 'exchange': "('exchange')"}}
}

In [19]:
query_params['return_product_weekly']['view']['actual']

['SELECT fulfillment_week_ended week_ended, %s, %s qty_returned, COUNT(*) qty_sold, %s * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1%s',
 'SUM(CASE WHEN send_back__type in %s THEN 1 ELSE 0 END)']

In [54]:
for key, value in query_params.items():
    '''Add logic to populate each tab in google sheet'''
    tab_df = pd.DataFrame()
    combinations = []
    keys = value.keys()
    values = (value[key] for key in keys)
    combinations.append(dict(zip(keys, combination)) for combination in itertools.product(*values))
    for combination in combinations:
        for combo in combination:
            keys = list(keys)
            grouping = ""
            for i in range(len(keys)):
#                 print(query_params[key][keys[i]][combo[keys[i]]])
                if i == 0:
                    query = query_params[key][keys[i]][combo[keys[i]]][0]
#                     print(query)
                    calculation = query_params[key][keys[i]][combo[keys[i]]][1]
#                     print(calculation)
                    params = f"'{combo[keys[i]]}' AS {keys[i]}"
#                     print(params)
                else:
                    grouping = f"{grouping}, {i+1}"
                    if keys[i] == 'send_back__type':
                        calculation = calculation %(str(query_params[key][keys[i]][combo[keys[i]]]))
                        params = f"{params}, '{combo[keys[i]]}' {keys[i]}"
                    else:
                        params = f"{params}, {query_params[key][keys[i]][combo[keys[i]]]}"
            query = query %(params, calculation, calculation, grouping)
#             print(query)
#             print(calculation)
#             print(params)
#             print(grouping)
            df = pd.read_sql(query, conn)
            tab_df = tab_df.append(df)
    

In [61]:
tab_df.iloc[2050]

week_ended                   2020-04-12
view                             actual
product            rectangular_chenille
send_back__type                exchange
qty_returned                          8
qty_sold                          14167
rate                        0.000564693
Name: 138, dtype: object